In [152]:
import pandas as pd
import numpy as np
import glob
import sys
import os

In [153]:
n = [25, 50, 75, 100]
pm = [0, 0.01, 0.03, 0.05]
pc = [0, 0.1, 0.3, 0.5]
trn = [2,3,4,5]

In [188]:

file_list = []

for N in n:
    for m in pm:
        for c in pc:
            for t in trn:
                for i in range(1,21):
                    file_list.append(f'n{N}_pm{m}_pc{c}_t{t}_itr{i}.csv')


#file_list = glob.glob(os.path.join(os.getcwd(), "*.csv"))
#file_list = [f.replace(os.getcwd()+'/', '') for f in file_list]
dfs = [pd.read_csv(f) for f in file_list]
df = pd.concat(dfs, ignore_index=True)

In [189]:
print(df.head(50))

    step                 fitness  \
0      0    0.012383741664729863   
1      0    0.015868218029444707   
2      0    0.012957497491787113   
3      0    0.012957497491787113   
4      0     0.02828486374834793   
5      0     0.22107474421499906   
6      0     0.02828486374834793   
7      0    0.015868218029444707   
8      0     0.22107474421499906   
9      0     0.33513976074415414   
10     0     0.02828486374834793   
11     0  1.9929132225402484e-05   
12     0     0.02828486374834793   
13     0   0.0011814929253461735   
14     0     0.42319928710854066   
15     0     0.01489420265195866   
16     0     0.22107474421499906   
17     0      0.4761855975654958   
18     0    0.015868218029444707   
19     0   0.0003986082106849183   
20     0      0.4761855975654958   
21     0     0.42319928710854066   
22     0   0.0011814929253461735   
23     0    0.012383741664729863   
24     0    0.015868218029444707   
25  step                 fitness   
26     1      0.476185597565

In [190]:
df_t_dirty = pd.read_csv("n25_pm0_pc0_t2_itr1.csv")
df_t = df_t_dirty[df_t_dirty['step'] != 'step']
df_t = df_t_dirty[df_t_dirty['fitness'] != 'fitness']
df_t = df_t_dirty[df_t_dirty['genome'] != 'genome']
df_t = df_t.apply(pd.to_numeric, errors='ignore')
df_t = df_t.reset_index(drop=True)


/tmp/ipykernel_627309/150920010.py:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_t = df_t.apply(pd.to_numeric, errors='ignore')


In [265]:
from scipy.spatial.distance import pdist
def calc_diversity(genomes):
    matrix = np.stack(genomes['genome'].values)
    distances = pdist(matrix, metric='euclidean')
    if len(distances) == 0: return 0.0
    return np.mean(distances)

In [ ]:
import ast
def get_stats_for_file(filename):
    df_t_dirty = pd.read_csv(filename)
    file_split = filename.split('_')

    n_val = int(file_split[0][file_split[0].find('n')+1 :])
    pm_val = float(file_split[1][file_split[1].find('m')+1 :])
    pc_val = float(file_split[2][file_split[2].find('c')+1 :])
    t_val = int(file_split[3][file_split[3].find('t')+1 :])
    itr_val = int(file_split[4][file_split[4].find('r')+1 : file_split[4].find('.')])

    
    df_t = df_t_dirty[df_t_dirty['step'] != 'step']
    df_t = df_t[df_t['fitness'] != 'fitness']
    df_t = df_t[df_t['genome'] != 'genome']
    df_t = df_t.apply(pd.to_numeric, errors='ignore')
    df_t = df_t.reset_index(drop=True)
    
    df_t['genome'] = df_t['genome'].apply(ast.literal_eval)
    df_t['genome'] = df_t['genome'].apply(lambda x: [int(i) for i in x])
    
    
    stats = df_t.groupby('step')['fitness'].agg(['mean', 'max'])
    stats = stats.rename(columns={'mean': 'Average Fitness', 'max': 'Best Fitness'})
   
    
    diversity = df_t.groupby('step').apply(calc_diversity)
    stats['Diversity Metric'] = diversity


    idx_of_max = df_t.groupby('step')['fitness'].idxmax()
    stats['Best Genome'] = df_t.loc[idx_of_max].set_index('step')['genome']

    ones = (df_t['fitness'] == 1).groupby(df_t['step']).sum()
    stats['Solution Count'] = ones
    stats['Solution Found'] = (stats['Solution Count'] != 0).astype(int)
    
    stats['Generation'] = stats.index
    stats['Iteration'] = itr_val
    stats['Tournament Size'] = t_val
    stats['P_c'] = pc_val
    stats['P_m'] = pm_val
    stats['N'] = n_val
    
    # --- 8. Final Output ---
    cols = ['N', 'P_m', 'P_c', 'Tournament Size', 'Iteration', 'Generation', 
            'Average Fitness', 'Best Fitness', 'Solution Found', 'Solution Count', 
            'Best Genome', 'Diversity Metric']
    
    # Safely select columns that exist
    stats = stats[[c for c in cols if c in stats.columns]]
    
    return stats
   
    

In [277]:

df_total = pd.DataFrame()

#x = get_stats_for_file('n100_pm0.01_pc0.1_t2_itr1.csv')
for f in file_list:
    df_total = pd.concat([df_total, get_stats_for_file(f)], ignore_index=True)
#print(x)
#x.to_csv('test.csv',index=False)


/tmp/ipykernel_627309/2552072059.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_t = df_t.apply(pd.to_numeric, errors='ignore')
/tmp/ipykernel_627309/2552072059.py:31: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  diversity = df_t.groupby('step').apply(calc_diversity)
/tmp/ipykernel_627309/2552072059.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_t = df_t.apply(pd.to_numeric, errors='ignore')
/tmp/ipykernel_627309/2552072059.py:31: FutureWarning: DataFram

In [278]:
df_total = df_total.reset_index(drop=True)
print(df_total)
df_total.to_csv("master.csv", index=False)

          N   P_m  P_c  Tournament Size  Iteration  Generation  \
0        25  0.00  0.0                2          1           0   
1        25  0.00  0.0                2          1           1   
2        25  0.00  0.0                2          1           2   
3        25  0.00  0.0                2          1           3   
4        25  0.00  0.0                2          1           4   
...     ...   ...  ...              ...        ...         ...   
153595  100  0.05  0.5                5         20          25   
153596  100  0.05  0.5                5         20          26   
153597  100  0.05  0.5                5         20          27   
153598  100  0.05  0.5                5         20          28   
153599  100  0.05  0.5                5         20          29   

        Average Fitness  Best Fitness  Solution Found  Solution Count  \
0              0.121684      0.476186               0               0   
1              0.218215      0.476186               0        

In [261]:
print((file_list))

['n25_pm0_pc0_t2_itr1.csv', 'n25_pm0_pc0_t2_itr2.csv', 'n25_pm0_pc0_t2_itr3.csv', 'n25_pm0_pc0_t2_itr4.csv', 'n25_pm0_pc0_t2_itr5.csv', 'n25_pm0_pc0_t2_itr6.csv', 'n25_pm0_pc0_t2_itr7.csv', 'n25_pm0_pc0_t2_itr8.csv', 'n25_pm0_pc0_t2_itr9.csv', 'n25_pm0_pc0_t2_itr10.csv', 'n25_pm0_pc0_t2_itr11.csv', 'n25_pm0_pc0_t2_itr12.csv', 'n25_pm0_pc0_t2_itr13.csv', 'n25_pm0_pc0_t2_itr14.csv', 'n25_pm0_pc0_t2_itr15.csv', 'n25_pm0_pc0_t2_itr16.csv', 'n25_pm0_pc0_t2_itr17.csv', 'n25_pm0_pc0_t2_itr18.csv', 'n25_pm0_pc0_t2_itr19.csv', 'n25_pm0_pc0_t2_itr20.csv', 'n25_pm0_pc0_t3_itr1.csv', 'n25_pm0_pc0_t3_itr2.csv', 'n25_pm0_pc0_t3_itr3.csv', 'n25_pm0_pc0_t3_itr4.csv', 'n25_pm0_pc0_t3_itr5.csv', 'n25_pm0_pc0_t3_itr6.csv', 'n25_pm0_pc0_t3_itr7.csv', 'n25_pm0_pc0_t3_itr8.csv', 'n25_pm0_pc0_t3_itr9.csv', 'n25_pm0_pc0_t3_itr10.csv', 'n25_pm0_pc0_t3_itr11.csv', 'n25_pm0_pc0_t3_itr12.csv', 'n25_pm0_pc0_t3_itr13.csv', 'n25_pm0_pc0_t3_itr14.csv', 'n25_pm0_pc0_t3_itr15.csv', 'n25_pm0_pc0_t3_itr16.csv', 'n25_pm0_